In [336]:
from sklearn.datasets import load_iris
from keras.utils import to_categorical
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# Data Preprocessing

In [337]:
iris = load_iris()
x_data=iris.data
y_data=iris.target

x_data = np.array(x_data, dtype=np.float32)
train_x, test_x, train_y, test_y = train_test_split(x_data, y_data)

In [350]:
class IrisDataset(Dataset):
    def __init__(self, x, y):
        super(IrisDataset).__init__()
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return torch.from_numpy(np.array(self.x[index])), torch.from_numpy(np.array(self.y[index]))
    def __len__(self):
        return len(self.x)

In [354]:
def calculate_accuracy(outputs, targets):
    batch_size = targets.size(0)

    _, pred = outputs.topk(1, 1, True)
    pred = pred.t()
    correct = pred.eq(targets.view(1, -1))
    n_correct_elems = correct.float()[0].sum().data

    return n_correct_elems / batch_size

# Model

In [352]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.l1 = torch.nn.Linear(4, 10)
        self.l2 = torch.nn.Linear(10, 3)
        self.sigmoid=torch.nn.Sigmoid()
        self.softmax = torch.nn.Softmax()
        self.relu=torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.2, inplace=False)
    def forward(self,x):
        out1=self.relu(self.l1(x))
        drop1=self.dropout(out1)
        out2=self.l2(drop1)
        y_pred = self.softmax(out2)
        return y_pred

# Training with SGD

In [351]:
batch_size = 50
train_dataset = IrisDataset(train_x, train_y)
test_dataset = IrisDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [353]:
model =Model()
criterion=torch.nn.CrossEntropyLoss()
opt= torch.optim.SGD(model.parameters(),lr=0.04)

In [355]:
dfor epoch in range(90):
    for step in ['train', 'test']:
        print(f'Epoch: {epoch} [{step.capitalize()}]')
        accuracies = []
        losses = []
        if step == 'train':
            loader = train_loader
            model.train()
        else:
            loader = test_loader
            model.eval()
        for i, (data, target) in enumerate(loader):
            inputs = Variable(data)
            outputs = Variable(target)
            y_pred_val=model(inputs)
            acc = calculate_accuracy(y_pred_val, outputs)
            accuracies.append(acc)
            loss=criterion(y_pred_val, outputs)
            losses.append(loss.data)
            if step == 'train':
                opt.zero_grad()
                loss.backward()
                opt.step()
        print(f'Loss: {np.sum(losses)/len(loader)}', f'Acc: {np.sum(accuracies)/len(loader)}')


Epoch: 0 [Train]
Loss: 1.0429449081420898 Acc: 0.5966666539510092
Epoch: 0 [Test]
Loss: 1.0101237297058105 Acc: 0.7368420958518982
Epoch: 1 [Train]
Loss: 1.0379211107889812 Acc: 0.5600000222524008
Epoch: 1 [Test]
Loss: 0.998529851436615 Acc: 0.7368420958518982
Epoch: 2 [Train]
Loss: 1.0039017995198567 Acc: 0.5733333428700765
Epoch: 2 [Test]
Loss: 0.9888100624084473 Acc: 0.7368420958518982
Epoch: 3 [Train]
Loss: 1.0270059903462727 Acc: 0.5333333412806193
Epoch: 3 [Test]
Loss: 0.9821146726608276 Acc: 0.7894737124443054
Epoch: 4 [Train]
Loss: 1.0092992782592773 Acc: 0.5133333206176758
Epoch: 4 [Test]
Loss: 0.9741047024726868 Acc: 0.7631579041481018
Epoch: 5 [Train]
Loss: 0.9878815015157064 Acc: 0.601111094156901
Epoch: 5 [Test]
Loss: 0.9686558246612549 Acc: 0.7894737124443054
Epoch: 6 [Train]
Loss: 0.9831022421518961 Acc: 0.602222204208374
Epoch: 6 [Test]
Loss: 0.9592795372009277 Acc: 0.7631579041481018
Epoch: 7 [Train]
Loss: 0.9919508298238119 Acc: 0.5455555518468221
Epoch: 7 [Test]
Loss

/Users/elizavetabatanina/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Loss: 0.8670772910118103 Acc: 0.9210526347160339
Epoch: 28 [Train]
Loss: 0.8938786188761393 Acc: 0.7366666793823242
Epoch: 28 [Test]
Loss: 0.8593478202819824 Acc: 0.9210526347160339
Epoch: 29 [Train]
Loss: 0.9221169153849283 Acc: 0.6677777767181396
Epoch: 29 [Test]
Loss: 0.8742420077323914 Acc: 0.9210526347160339
Epoch: 30 [Train]
Loss: 0.8975969950358073 Acc: 0.7911110719045004
Epoch: 30 [Test]
Loss: 0.8579500913619995 Acc: 0.9210526347160339
Epoch: 31 [Train]
Loss: 0.9340558052062988 Acc: 0.7144443988800049
Epoch: 31 [Test]
Loss: 0.864666759967804 Acc: 0.9736841917037964
Epoch: 32 [Train]
Loss: 0.889604647954305 Acc: 0.7766666412353516
Epoch: 32 [Test]
Loss: 0.8490470051765442 Acc: 0.9210526347160339
Epoch: 33 [Train]
Loss: 0.9124702612559 Acc: 0.6944444179534912
Epoch: 33 [Test]
Loss: 0.8553348183631897 Acc: 0.9473684430122375
Epoch: 34 [Train]
Loss: 0.8903622627258301 Acc: 0.7699999809265137
Epoch: 34 [Test]
Loss: 0.8591601848602295 Acc: 0.9210526347160339
Epoch: 35 [Train]
Loss: 0

# Simulated Annealing

In [375]:
class SimulatedAnnealing:
    def __init__(self, T, max_t, min_T, energy_xt, model):
        self.T = T
        self.t = 0
        self.params_shapes = [p.shape for p in model.parameters()]
        self.shape_x = sum([np.prod(sh) for sh in self.params_shapes])
        self.max_t = max_t
        self.min_T = min_T
        x0 = self.extract_parameters(model)
        self.x_t = x0
        self.x_prime = x0
        self.energy_xt =  energy_xt
        self.alpha = 0.99 # exponential?
        self.best_position = (energy_xt, x0)
    def accept(self, energy_x):
        alpha = np.random.uniform(0, 1)
        if self.__prob(self.energy_xt) == 0.0:
            return True
        ratio = self.__prob(energy_x)/ self.__prob(self.energy_xt)
        if ratio >= alpha:
            return True
        return False
    def step(self, model, energy_x):
        if self.accept(energy_x):
            self.x_t = self.x_prime
            self.t += 1
            if energy_x < self.best_position[0]:
                self.best_position = (energy_x, self.x_prime)
            if self.t > self.max_t:
                self.T *= self.alpha
                self.t = 0
        self.generate() # new x_prime
        self.set_parameters(model, self.x_prime) # set new x_prime_params
        
    def extract_parameters(self, model):
        array = np.concatenate([p.data.flatten() for p in model.parameters()])
        return array
    def set_parameters(self, model, x):
        shapes = [p.shape for p in model.parameters()]
        lengths = [np.prod(sh) for sh in shapes]
        cuts_ids = [lengths[0]]
        for i in range(1, len(lengths)):
            cuts_ids.append(cuts_ids[i-1] + lengths[i])
        cuts = np.array_split(x, cuts_ids)
        new_params = []
        for i, param in enumerate(model.parameters()):
            param.data = torch.tensor(np.reshape(cuts[i], shapes[i]), dtype=torch.float32)
        
    def __prob(self, energy_x):
        return (np.e)**(-energy_x/self.T)
    
    def generate(self):
        self.x_prime = np.random.normal(loc=np.mean(self.x_t), size=self.shape_x).astype(np.float32)
        

In [376]:
model =Model()
y_pred_val=model(Variable(torch.Tensor(train_x)))
loss=criterion(y_pred_val, torch.from_numpy(train_y))
sa_opt = SimulatedAnnealing(float(loss.data)*2, 60, 0.01, float(loss.data), model)

/Users/elizavetabatanina/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


In [377]:
batch_size = 50
train_dataset = IrisDataset(train_x, train_y)
test_dataset = IrisDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [378]:
N = 20000
for epoch in range(N):
    for step in ['train', 'test']:
        
        accuracies = []
        losses = []
        if step == 'train':
            loader = train_loader
            model.train()
        else:
            loader = test_loader
            model.eval()
        for i, (data, target) in enumerate(loader):
            inputs = Variable(data)
            outputs = Variable(target)
            y_pred_val=model(inputs)
            acc = calculate_accuracy(y_pred_val, outputs)
            accuracies.append(acc)
            loss=criterion(y_pred_val, outputs)
            losses.append(loss.data)
            if step == 'train':
                sa_opt.step(model, float(loss.data))
        if (epoch % 600 == 0) or (epoch==(N-1)):
            print(f'Epoch: {epoch} [{step.capitalize()}]')
            print(f'T:{sa_opt.T}, Best: {sa_opt.best_position[0]}, t: {sa_opt.t}')
            print(f'Loss: {np.sum(losses)/len(loader)}', f'Acc: {np.sum(accuracies)/len(loader)}')
            print('================================================================================')
#         print()

/Users/elizavetabatanina/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Epoch: 0 [Train]
T:2.2771453857421875, Best: 1.1382832527160645, t: 3
Loss: 1.1689612865447998 Acc: 0.32555556297302246
Epoch: 0 [Test]
T:2.2771453857421875, Best: 1.1382832527160645, t: 3
Loss: 1.0835131406784058 Acc: 0.42105263471603394
Epoch: 600 [Train]
T:1.7186055423863842, Best: 0.8344288468360901, t: 42
Loss: 1.0693312486012776 Acc: 0.4144444465637207
Epoch: 600 [Test]
T:1.7186055423863842, Best: 0.8344288468360901, t: 42
Loss: 1.1073907613754272 Acc: 0.3684210479259491
Epoch: 1200 [Train]
T:1.2840941024347232, Best: 0.8344288468360901, t: 54
Loss: 1.1597588857014973 Acc: 0.25555557012557983
Epoch: 1200 [Test]
T:1.2840941024347232, Best: 0.8344288468360901, t: 54
Loss: 1.1665098667144775 Acc: 0.2631579041481018
Epoch: 1800 [Train]
T:0.9594392798350035, Best: 0.8344288468360901, t: 3
Loss: 1.2008606592814128 Acc: 0.352222204208374
Epoch: 1800 [Test]
T:0.9594392798350035, Best: 0.8344288468360901, t: 3
Loss: 1.0468164682388306 Acc: 0.5
Epoch: 2400 [Train]
T:0.7388096412531788, Bes

Epoch: 12000 [Train]
T:0.011637720509708328, Best: 0.7301239967346191, t: 57
Loss: 1.110044797261556 Acc: 0.38555554548899335
Epoch: 12000 [Test]
T:0.011637720509708328, Best: 0.7301239967346191, t: 57
Loss: 1.1108384132385254 Acc: 0.3684210479259491
Epoch: 12600 [Train]
T:0.009518572342708021, Best: 0.7301239967346191, t: 20
Loss: 1.110204855600993 Acc: 0.34555554389953613
Epoch: 12600 [Test]
T:0.009518572342708021, Best: 0.7301239967346191, t: 20
Loss: 1.1138323545455933 Acc: 0.3684210479259491
Epoch: 13200 [Train]
T:0.007554074971068538, Best: 0.7301239967346191, t: 6
Loss: 1.127679745356242 Acc: 0.34555554389953613
Epoch: 13200 [Test]
T:0.007554074971068538, Best: 0.7301239967346191, t: 6
Loss: 1.1037757396697998 Acc: 0.2631579041481018
Epoch: 13800 [Train]
T:0.006055577572479822, Best: 0.7301239967346191, t: 0
Loss: 1.0984560648600261 Acc: 0.36666667461395264
Epoch: 13800 [Test]
T:0.006055577572479822, Best: 0.7301239967346191, t: 0
Loss: 1.151438593864441 Acc: 0.2631579041481018


## Evaulation of best parameters

In [382]:
sa_opt.set_parameters(model, sa_opt.best_position[1])

In [383]:
y_pred_val=model(Variable(torch.Tensor(train_x)))
print(calculate_accuracy(y_pred_val, Variable(torch.Tensor(train_y))))
y_pred_val=model(Variable(torch.Tensor(test_x)))
print(calculate_accuracy(y_pred_val, Variable(torch.Tensor(test_y))))

tensor(0.6607)
tensor(0.6053)


/Users/elizavetabatanina/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
